In [ ]:
import os
import random

from catboost import CatBoostClassifier

import dateutil.relativedelta as relativedelta


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web

from scipy import stats

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

import seaborn as sns
from typing import Any, List, Optional, Union, Tuple


In [ ]:
# set N used in n-largest or smallest
N = 10


In [ ]:
# set style
plt.style.use("seaborn-notebook")

# set ratio of figure
ratio = (16, 9)


In [ ]:
# set fixed seed
def seed_everything(seed) -> None:
    """
    Seeds basic parameters for reproducibility of results.
    """
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    # pandas and numpy as discussed here: https://stackoverflow.com/a/52375474/5755604
    np.random.seed(seed)


seed = 42
seed_everything(seed)


This notebook performs an eda on the training set only to avoid data leakage. ⚠️

In [ ]:
data = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_extended_60.parquet"
)


In [ ]:
data = data.sample(frac=0.1, axis=0, random_state=seed)


## Notes on data set 🗃️

**Overview on ticker symbols:**
- `others` identified by issue type.
- 5th letter has a special meaning as found in [this table](https://en.wikipedia.org/wiki/Ticker_symbol):

| Letter                  | Letter contd.              | Letter contd.                                    |
|--------------------------------|-------------------------------------|------------------------------------------------|
| A – Class "A"                  | K – Nonvoting (common)              | U – Units                                      |
| B – Class "B"                  | L – Miscellaneous                   | V – Pending issue and distribution             |
| C – NextShares                 | M – fourth class – preferred shares | W – Warrants                                   |
| D – New issue or reverse split | N – third class – preferred shares  | X – Mutual fund                                |
| E – Delinquent SEC filings     | O – second class – preferred shares | Y – American depositary receipt (ADR)          |
| F – Foreign                    | P – first class preferred shares    | Z – Miscellaneous situations                   |
| G – first convertible bond     | Q – In bankruptcy                   | Special codes                                  |
| H – second convertible bond    | R – Rights                          | PK – A Pink Sheet, indicating over-the-counter |
| I – third convertible bond     | S – Shares of beneficial interest   | SC – Nasdaq Small Cap                          |
| J – Voting share – special     | T – With warrants or rights         | NM – Nasdaq National Market                    |


**Coverage:**

*	Options on U.S. listed Stock, ETFs, and Indices disseminated over the Options Price Reporting Authority (OPRA) market data feed 
*	Global Trading Hours (GTH) trades are included if between 03:00am-09:15am U.S. Eastern, and for the 16:15pm 17:00pm Curb session.  GTH trades outside of these time ranges will *not* be included. 

Found [here.](https://datashop.cboe.com/documents/Option_Trades_Layout.pdf)

**Exchange Identifier:**

- 5 = Chicago Board Options Exchange (CBOE)
- 6 = International Securities Exchange (ISE)

Found [here.](https://datashop.cboe.com/documents/livevol_exchange_ids.csv)


**Issue Types:**

Issue Type = the type of security: 
- 0 = Common Stock 
- A = Market index 
- 7 = Mutual or investment trust fund 
- F = ADR/ADS 
- % = Exchange-traded fund 
- (blank) = Unspecified

Received from supervisor.

Adapted from the cboe data shop found at [option trades](https://datashop.cboe.com/documents/Option_Trades_Layout.pdf) and [option quotes](https://datashop.cboe.com/documents/Option_Quotes_Layout.pdf).

|     Column Label                                                          |     Data   Type     |     Description                                                                                                                                                                                                         |
|---------------------------------------------------------------------------|---------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|     underlying_symbol                                                     |     string          |     The underlying stock or index.  An index will utilize a caret (^) prefix,   i.e. ^NDX,^SPX,^VIX…etc.  Underlyings   with classes may utilize a dot (.) instead of a slash or space, i.e. BRK.B,   RDS.A, RDS.B.     |
|     quote_datetime                                                        |     datetime        |     The trading date and timestamp of the trade in   U.S. Eastern time. Ex:  yyyymm-dd   hh:mm:ss.000                                                                                                                   |
|     sequence_number                                                       |     integer         |     Trade Sequence Number for the execution reported   by OPRA                                                                                                                                                          |
|     root                                                                  |     string          |     The option trading class symbol.  Non-standard roots may end with a digit                                                                                                                                           |
|     expiration                                                            |     date            |     The explicit expiration date of the option:   yyyy-mm-dd                                                                                                                                                            |
|     strike                                                                |     numeric         |     The exercise/strike price of the option                                                                                                                                                                             |
|     option_type                                                           |     string          |     C for Call options, P for Put options                                                                                                                                                                               |
|     exchange_id                                                           |     integer         |     An identifier for the options exchange the trade   was executed on.  For a mapping, please   see Exchange ID   Mappings                                                                                             |
|     trade_size                                                            |     integer         |     The trade quantity                                                                                                                                                                                                  |
|     trade_price                                                           |     numeric         |     The trade price                                                                                                                                                                                                     |
|     trade_condition_id                                                    |     integer         |     The trade or sale condition of the execution.  For a mapping, please see Trade   Condition ID Mapping                                                                                                               |
|     canceled_trade_condition_id                                           |     integer         |     This field is no longer supported and will default   to 0 (zero).  See IDs 40-43 in the   Trade Condition ID Mapping file above                                                                                     |
|     best_bid                                                              |     numeric         |     The best bid price (NBB) at the time of the trade                                                                                                                                                                   |
|     best_ask                                                              |     numeric         |     The best ask/offer price (NBO) at the time of the   trade                                                                                                                                                           |
|     bid_size              |     integer         |     The largest size from an options exchange   participant on the best bid price (NBB)                                                                                                                                   |
|     bid                   |     numeric         |     The best bid price (NBB) at the interval time   stamp                                                                                                                                                                 |
|     ask_size              |     integer         |     The largest size from an options exchange   participant on the best offer price (NBO)                                                                                                                                 |
|     ask                   |     numeric         |     The best offer price (NBO) at the interval time   stamp                                                                                                                                                               |

## Dtypes, distributions, and memory consumption 🔭

In [ ]:
data.head()


In [ ]:
data.describe()


In [ ]:
data.info()


In [ ]:
print(data.shape)


In [ ]:
print(data.shape)
# drop identical rows, if present
data.drop_duplicates(inplace=True)
print(data.shape)


**Observation:**
- Shape matches the shape reported in table 1 (panel A) of Grauer et al. paper.
- No duplicates

In [ ]:
data.nunique()


In [ ]:
data.head().T


## Basic features🧸

Analysis of numerical features without any feature engineering.

### Correlations 🎲

In [ ]:
corr: pd.DataFrame = data.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values)  # type: ignore


**Observation:**
* There are many highly correlated columns. The correlations are intuitive e. g., between `price_all_lead` and `price_ex_lead`.
* Few columns show a weak correlation with target (see also below).

In [ ]:
sample = data.sample(n=1000, random_state=seed)
sns.pairplot(
    sample,
    vars=[
        "STRK_PRC",
        "TRADE_SIZE",
        "TRADE_PRICE",
        "BEST_BID",
        "BEST_ASK",
        "ask_ex",
        "bid_ex",
        "ask_size_ex",
        "bid_size_ex",
        "price_all_lag",
        "price_all_lead",
        "day_vol",
    ],
)


### Correlation with target 🎲

In [ ]:
sort_criteria = corr["buy_sell"].abs().sort_values(ascending=False)
corr_target = corr.sort_values("buy_sell", ascending=False)["buy_sell"]
corr_target.loc[sort_criteria.index].to_frame()


**Observation:**
* Overall correlations are relatively low. Typical for financial data due to low signal-to-noise ratio.
* Size-related features like `ask_size_ex` or `bid_size_ex` have the highest correlation with the target. Thus, can be promising to be included in the model. Consider size features when constructing feature sets.
* Features like `optionid`, `order_id`, and `SEQUENCE_NUMBER` are also among the features with the highest correlations. Remove them, as the correlation is misleading.

In [ ]:
# remove some columns, which will NOT be used in model
data.drop(columns=["optionid"], inplace=True)


### Collinearity of features 🎲

In [ ]:
# adapted from here: https://www.kaggle.com/code/willkoehrsen/featuretools-for-good
threshold = 0.975
# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.975
to_drop = [column for column in upper.columns if any(abs(upper[column]) > threshold)]

print(to_drop)


**Observation:**
- Columns suggested for removal are intuitive
- Do not blindly remove columns, but preserve a pattern or groups

In [ ]:
# Set the threshold
threshold = 0.975

# Empty dictionary to hold correlated variables
above_threshold_vars = {}

# For each column, record the variables that are above the threshold
for col in corr:
    above_threshold_vars[col] = list(corr.index[corr[col] > threshold])


In [ ]:
pd.Series(above_threshold_vars)


**Observations:**
* Some columns are highly correlated. This is very intuitive.
* It seems problematic to include both `BEST_BID` and `bid_ex`. This is also true for `BEST_ASK` and `ask_ex`. `price_all_lead` and `price_all_lag` seem to be less problematic.
* Define feature sets so that the number of highly correlated variables is minimized. But maintain groups so that a comparsion with classical rules is still possible.

## Preparation 🥗

### Time features ⏰

In [ ]:
# apply positional encoding to dates
data["date_month_sin"] = np.sin(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)
data["date_month_cos"] = np.cos(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)

# time (daily)
seconds_in_day = 24 * 60 * 60
seconds = (
    data["QUOTE_DATETIME"] - data["QUOTE_DATETIME"].dt.normalize()
).dt.total_seconds()

data["date_time_sin"] = np.sin(2 * np.pi * seconds / seconds_in_day)
data["date_time_cos"] = np.cos(2 * np.pi * seconds / seconds_in_day)

# year min-max scaled
data["date_year_min"] = (data["QUOTE_DATETIME"].dt.year - 2005) / (2017 - 2005)

# time to maturity
data["ttm"] = (
    data["EXPIRATION"].dt.to_period("M") - data["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

# day, month and year
data["day"] = data["QUOTE_DATETIME"].dt.day
data["month"] = data["QUOTE_DATETIME"].dt.month
data["year"] = data["QUOTE_DATETIME"].dt.year
data["date"] = data["QUOTE_DATETIME"].dt.date


### Binned features 🥫

Bin features similarily to how they are used in the robustness tests.

In [ ]:
bins_tradesize = [0, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
data["TRADE_SIZE_binned"] = pd.cut(
    data["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
data["year_binned"] = pd.cut(data["year"], bins_years, labels=year_labels)

bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
data["ttm_binned"] = pd.cut(data["ttm"], bins_ttm, labels=ttm_labels)


### Trade features 💴
Construct features that are used in classical rules.

In [ ]:
# spread in $ between ask and bid
data["spread_ex"] = data["ask_ex"] - data["bid_ex"]

# Calculate change similar to tick rule
data["chg_lead_ex"] = data["TRADE_PRICE"] - data["price_ex_lead"]

# Calculate change similar to reverse tick rule
data["chg_lag_ex"] = data["TRADE_PRICE"] - data["price_ex_lag"]

# Midspread
mid_ex = 0.5 * (data["ask_ex"] + data["bid_ex"])
mid_best = 0.5 * (data["BEST_ASK"] + data["BEST_BID"])

# ratio of bid-ask
data["bid_ask_ratio_ex"] = data["bid_ex"] / data["ask_ex"]

# Absolute distance from mid
data["abs_mid_ex"] = data["TRADE_PRICE"] - mid_ex
data["mid_ex"] = mid_ex

# Absolute distance from mid
data["abs_mid_BEST"] = data["TRADE_PRICE"] - mid_best
data["mid_best"] = mid_best

# depth rule (usually only applied to mid spread transactions)
data["bid_ask_size_ratio_ex"] = data["bid_size_ex"] / data["ask_size_ex"]

# Degree how much trade size is filled -> similar to trade size rule
# Trade size rule would just classify if
data["rel_bid_size_ex"] = data["TRADE_SIZE"] / data["bid_size_ex"]
data["rel_ask_size_ex"] = data["TRADE_SIZE"] / data["ask_size_ex"]

# EMO / CLNV
data["rel_ask_ex"] = (data["TRADE_PRICE"] - mid_ex) / (data["ask_ex"] - mid_ex)
data["rel_bid_ex"] = (mid_ex - data["TRADE_PRICE"]) / (mid_ex - data["bid_ex"])

# EMO / CLNV
data["BEST_rel_bid"] = (data["TRADE_PRICE"] - mid_best) / (data["BEST_ASK"] - mid_best)
data["BEST_rel_ask"] = (mid_best - data["TRADE_PRICE"]) / (mid_best - data["BEST_BID"])


### Underlying features 🫀

In [ ]:
data["symbol_is_index"] = data["ROOT"].str.startswith("^").astype(int)


# TODO: Add majority class @ day


### Categorical features 🎰

### Visualization helper 🐜

In [ ]:
def plot_kde_target(var_name: str, clip: Optional[List[float]] = None):
    """
  Plot kde plots for buys (+1) and sells (-1) with regard to \
  the feature 'var_name'.

   Args:
      var_name (str): name of the feature
      clip (Optional[List[float]], optional): clipping range. Defaults to None.
  """
    corr_var = data["buy_sell"].corr(data[var_name])

    median_sell = data[data["buy_sell"] == -1][var_name].median()
    median_buy = data[data["buy_sell"] == 1][var_name].median()

    _, ax = plt.subplots()
    for i in [-1, 1]:
        sns.kdeplot(
            data=data[data["buy_sell"] == i],
            x=var_name,
            clip=clip,
            label=str(i),
            cumulative=False,
            common_grid=True,
        )
    ax.title.set_text(f"Distribution of '{var_name}'")
    ax.legend()
    sns.move_legend(ax, "lower center", bbox_to_anchor=(0.5, -0.3))
    plt.show()
    print(
        f"The correlation between '{var_name}' and the 'buy_sell' is {corr_var: 0.4f}"
    )
    print(f"Median value of sells = {median_sell: 0.4f}")
    print(f"Median value of buys = {median_buy: 0.4f}")


In [ ]:
def plot_kde_target_comparsion(
    var_name: str,
    clip: Optional[List[float]] = None,
    years: List[int] = [2006, 2010, 2013],
) -> None:
    """
    Plot several kde plots side by side for the feature.

    Args:
        var_name (str): name of the feature
        clip (Optional[List[float]], optional): clipping range. Defaults to None.
        years (List[int], optional): years to compare. Defaults to [2006, 2010, 2013].
    """
    fig, ax = plt.subplots(nrows=1, ncols=len(years), figsize=(18, 4))

    fig.suptitle(f"Distribution of `{var_name}`")

    for y, year in enumerate(years):
        for i in [-1, 1]:
            sns.kdeplot(
                data=data[(data["buy_sell"] == i) & (data["year"] == year)],
                x=var_name,
                clip=clip,
                # supress any other but first label using '_'
                # see https://stackoverflow.com/a/44633022/5755604
                label="_" * y + str(i),
                cumulative=False,
                common_grid=True,
                ax=ax[y],
            )
            ax[y].xaxis.label.set_text(str(year))

    fig.legend()


In [ ]:
us_rec = web.DataReader("USREC", "fred", data["date"].min(), data["date"].max())


def plot_recessions() -> None:
    """
    Add recession indicator to plot and entry to legend.
    """
    l = 0
    month = relativedelta.relativedelta(months=+1)
    for date, val in us_rec["USREC"].items():
        if val == 1:
            # if boolean = 1 -> print bar until next month
            # '_' labels are ignored in legend https://stackoverflow.com/a/44633022/5755604
            plt.axvspan(
                date,
                date + month,
                edgecolor="none",
                alpha=0.25,
                label="_" * l + "recession",
            )
            l += 1


In [ ]:
def plot_time_series(
    feature: Union[str, List[str]], aggregation: Union[str, List[Any]] = "count"
) -> pd.DataFrame:
    """
    Plot feature over time. Aggregate using 'aggregation'.

    Args:
        feature (Union[str, List[str]]): feature to plot.
        aggregation (Union[str, List[Any]], optional): aggregation operation. Defaults to "count".

    Returns:
        pd.DataFrame: time series
    """
    if isinstance(feature, str):
        feature = [feature]
    if isinstance(aggregation, str):
        aggregation = [aggregation]

    time_series = data[feature].groupby(data["date"]).agg(aggregation)
    time_series.columns = time_series.columns.to_flat_index()

    ax = sns.lineplot(data=time_series)
    ax.yaxis.label.set_text(" / ".join(aggregation))
    ax.title.set_text(f"'{' / '.join(feature)}' over time")
    plot_recessions()
    ax.legend()
    plt.show()

    return time_series


In [ ]:
# select categorical e. g., option type and strings e. g., ticker
cat_columns = data.select_dtypes(include=["category", "object"]).columns.tolist()
print(cat_columns)

# assign "bin_" column prefix
cat_columns_bin = ["bin_" + x for x in cat_columns]

# binarize categorical similar to Borisov et al.
data[cat_columns_bin] = data[cat_columns].apply(lambda x: pd.factorize(x)[0])  # type: ignore


## General overview 🌄

### Trade price and sizes 🤝

#### Trades over time ⌚

In [ ]:
trades_per_day = plot_time_series("TRADE_PRICE", "count")


In [ ]:
trades_per_day.iloc[:, 0].nlargest(N)


In [ ]:
trades_per_day.iloc[:, 0].nsmallest(N)


**Observation:**
* Number of trades increases over time.
* There is no obvious explanation why the number of trades spikes at certain days.

#### Trade size

In [ ]:
ax = sns.histplot(data, x="TRADE_SIZE", bins=50)  # type: ignore
ax.title.set_text("Histogram of trade size")


**Observation:**
* highly skewed with few outliers.
* Similar to the price, $\log(\cdot)$ transform could help.

In [ ]:
trades_over_time = plot_time_series("TRADE_SIZE", ["mean", "median"])


In [ ]:
trade_ask_bid_size = plot_time_series(
    ["TRADE_SIZE", "ask_size_ex", "bid_size_ex"], "mean"
)


**Observation:**
* There is a slow downward trend in `TRADE_SIZE` (mean).
* Controversely, the number of trades per day (mean) increases over time.
* Market share of ISE has decrease over time, as reported in https://www.sifma.org/wp-content/uploads/2022/03/SIFMA-Insights-Market-Structure-Compendium-March-2022.pdf. 

In [ ]:
data["TRADE_SIZE"].describe()


In [ ]:
data[data["TRADE_SIZE"].max() == data["TRADE_SIZE"]]


In [ ]:
data.nlargest(N, "TRADE_SIZE", keep="first").T


In [ ]:
data["log_trade_size"] = np.log1p(data["TRADE_SIZE"])
ax = sns.histplot(data, x="log_trade_size", bins=50)  # type: ignore
ax.title.set_text(f"Histogram of trade size (log1p)")


In [ ]:
plot_kde_target("log_trade_size", clip=[0, 6])


**Observation:**
- Size features do hardly profit from a $\log$ transform. Might want to keep as-is.

#### Trade price

In [ ]:
ax = sns.histplot(data, x="TRADE_PRICE", bins=50)  # type: ignore
ax.title.set_text("Histogram of trade price")


In [ ]:
ax = sns.boxplot(data=data, x="buy_sell", y="TRADE_PRICE")
ax.title.set_text("Box plot of 'TRADE_PRICE' for buys (1) and sells (-1)")


**Observations:**
* Very few, very large trade prices, many very small trade prices.
* Scaling can be problematic, if outliers affect scaling much. Try $\log(\cdot)$ transform to correct skewness of distribution. Could improve results.
* Trade price is hardly informative, as distribution is very similar.

In [ ]:
data["log_trade_price"] = np.log1p(data["TRADE_PRICE"])

In [ ]:
fig, ax = plt.subplots()

sns.histplot(data, x="log_trade_price", bins=50, stat="density", label="log price")  # type: ignore

# extract the limits for the x-axis and fit normal distributon
x0, x1 = ax.get_xlim()
x_pdf = np.linspace(x0, x1, 100)
y_pdf = stats.norm.pdf(x_pdf)

pdf = pd.DataFrame({"x": x_pdf, "y": y_pdf})
sns.lineplot(data=pdf, x="x", y="y", label="pdf", color="r")


ax.title.set_text("Distribution of log prices")
ax.legend()


In [ ]:
ax = sns.boxplot(data=data, x="buy_sell", y="log_trade_price")
ax.title.set_text("Box plot of log prices for buys (1) and sells (-1)")


In [ ]:
data.nlargest(N, "TRADE_PRICE", keep="first").T


In [ ]:
trade_price_over_time = plot_time_series("TRADE_PRICE", ["mean", "median"])


In [ ]:
trade_price_over_time = plot_time_series(
    ["TRADE_PRICE", "price_ex_lead", "price_ex_lag"], "mean"
)


In [ ]:
trade_price_over_time = plot_time_series(
    ["TRADE_PRICE", "price_ex_lead", "price_ex_lag"], "median"
)


**Observation:**
* `TRADE_PRICE` remains roughly constant over time. Median decreases over time.
* Large difference between median and mean. 

### Time to maturity ⌚

In [ ]:
ttm_over_time = plot_time_series("ttm", "mean")


In [ ]:
sample = data.sample(n=1000, random_state=seed)

plot = sns.displot(data=sample, x="ttm", y="TRADE_PRICE", kind="kde", hue="OPTION_TYPE")
plot.figure.subplots_adjust(top=0.9)
plot.figure.suptitle("Trade Price vs. Time to Maturity")


In [ ]:
ax = sns.scatterplot(data=sample, x="ttm", y="bid_ex", hue="OPTION_TYPE")
ax.title.set_text("Scatter plot of time to maturity (months) and bid (ex)")


In [ ]:
ax = sns.histplot(data=data[data["bid_ex"] == 0.0], x="ttm", bins=50)  # type: ignore
ax.title.set_text("Count of transactions with regard to time to maturity (months)")


**Observation:**
- Most options have a short time-to-maturity
- Binning or cut-off could be helpful

In [ ]:
# TODO: ask of zero plausible?
sns.histplot(data=data[data["ask_ex"] == 0.0], x="ttm", bins=50)  # type: ignore


### Strike price

In [ ]:
ax = sns.histplot(data, x="STRK_PRC", bins=50)  # type: ignore
ax.title.set_text("Histogram of strike price")


In [ ]:
ax = sns.boxplot(data=data, x="buy_sell", y="STRK_PRC")
ax.title.set_text("Box plot of strike prices for buys (1) and sells (-1)")


In [ ]:
strike_over_time = plot_time_series("STRK_PRC", "mean")


**Observation:**
- Distribution of strike price is highly skewed
- Average strike price grows over time. Thus, very large and previously unobserved trade prices could be part of the test set, but not in the train set.
- Try $\log$

In [ ]:
data["log_strk_prc"] = np.log1p(data["STRK_PRC"])


In [ ]:
ax = sns.histplot(data, x="log_strk_prc", bins=50)  # type: ignore
ax.title.set_text("Histogram of strike price (log1p)")


In [ ]:
ax = sns.boxplot(data=data, x="buy_sell", y="log_strk_prc")
ax.title.set_text("Box plot of strike prices for buys (1) and sells (-1)")


### Buy Sell 👛

In [ ]:
ratio_buy_sell = data["buy_sell"].value_counts() / data["buy_sell"].count()
ratio_buy_sell.head()


**Observation:**
* Ratios similar to the one reported in Grauer et. al. Yet not identical as calculation is done on a sample.
* As both classes have a $\approx~0.5$ probability, I would not rebalance. Rebalancing through sampling etc. itself has a bias.
* Ratios seem to be stable over time (see below). Thus, distribution is similar for training, validation, and test set.
* With regard to time-to-maturity the distribution changes slightly for longer periods.

#### By option type

In [ ]:
ax = sns.countplot(data=data, x="OPTION_TYPE", hue="buy_sell")
ax.title.set_text("Distribution of Buy / Sell indicator with regard to option type")
sns.move_legend(ax, "lower center", bbox_to_anchor=(0.5, -0.3))


#### By year

In [ ]:
ax = sns.countplot(data=data, x="year_binned", hue="buy_sell")
ax.title.set_text("Distribution of Buy / Sell indicator with regard to year (binned)")
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="center")
plt.tight_layout()
plt.show()


#### By time time to maturity

In [ ]:
ax = sns.countplot(data=data, x="ttm_binned", hue="buy_sell")
ax.title.set_text(
    "Distribution of Buy / Sell indicator with regard to time to maturity (binned)"
)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="center")
plt.tight_layout()
plt.show()


In [ ]:
ax = sns.scatterplot(data=sample, x="ttm", y="bid_ex", hue="OPTION_TYPE")
ax.title.set_text("Scatter plot of time to maturity (months) and bid (ex)")


In [ ]:
ax = sns.histplot(data=data[data["bid_ex"] == 0.0], x="ttm", bins=50)  # type: ignore
ax.title.set_text("Count of transactions with regard to time to maturity (months)")


In [ ]:
# TODO: ask of zero plausible?
sns.histplot(data=data[data["ask_ex"] == 0.0], x="ttm", bins=50)  # type: ignore


#### Over time

In [ ]:
trades_over_time = (
    data.groupby(data["date"])["buy_sell"].value_counts().unstack(fill_value=0)
)
ax = trades_over_time.plot(
    kind="line",
    figsize=ratio,
    title="buy / sell count over time",
    xlabel="date",
    ylabel="sell (-1) / buy (1)",
)
plot_recessions()
ax.legend()
plt.show()


### $n$ most frequent symbols, indices, and special codes 🔢

In [ ]:
most_frequent_symbols = data["ROOT"].value_counts().head(N).reset_index(name="Count")
most_frequent_symbols.rename(columns={"index": "Symbol"}, inplace=True)

ax = sns.barplot(data=most_frequent_symbols, x="Symbol", y="Count")
ax.title.set_text(f"{N} most frequently traded symbols")

most_frequent_symbols.head(N)


In [ ]:
list_freq_symbols = most_frequent_symbols.Symbol.tolist()


In [ ]:
frequent_symbols_over_time = data[data["ROOT"].isin(list_freq_symbols)]


In [ ]:
frequent_symbols_trades_per_day = (
    frequent_symbols_over_time.groupby(
        [frequent_symbols_over_time.QUOTE_DATETIME.dt.to_period("m"), "ROOT"]
    )["TRADE_SIZE"]
    .count()
    .reset_index()
    .rename(columns={"TRADE_SIZE": "count", "QUOTE_DATETIME": "date", "ROOT": "Symbol"})
)


In [ ]:
frequent_symbols_over_time = (
    frequent_symbols_trades_per_day.groupby(["date", "Symbol"])["count"]
    .first()
    .unstack()
)


In [ ]:
frequent_symbols_over_time.plot(
    kind="line", title=f"{N} most frequently traded underlyings over time"
)


In [ ]:
# TODO: investigate why there is no True group
ax = sns.countplot(data=data, x="symbol_is_index", hue="buy_sell")
ax.title.set_text(
    "Distribution of Buy / Sell indicator with regard to whether underlying is an index"
)
sns.move_legend(ax, "lower center", bbox_to_anchor=(0.5, -0.3))


In [ ]:
ratios_is_index = (
    data.groupby(["symbol_is_index", "buy_sell"])["buy_sell"].count()
    / data.groupby(["symbol_is_index"])["buy_sell"].count()
)
ratios_is_index.head()


**Observation:**
- Feature can be important, as it's much more likely for trade to be sell, rather than buy, if and only if the underlying is no index option.
- Difference isn't too pronounced and could be due to sampling effects.

In [ ]:
data["issue_type"].value_counts(dropna=False)


In [ ]:
ax = sns.countplot(data=data, x="issue_type")
ax.title.set_text("No. of transactions by issue type")
ax.xaxis.label.set_text("issue type")


###  Ask and bid👨‍⚖️

In [ ]:
bid_ask_over_time = plot_time_series(
    ["bid_ex", "ask_ex", "BEST_ASK", "BEST_BID"], "mean"
)


#### Ask

In [ ]:
ax = sns.histplot(data, x="ask_ex", bins=50)  # type: ignore
ax.title.set_text("Histogram of ask (exchange)")


**Observation:**
* Distribution is highly skewed, try correction with $\log$

In [ ]:
data["log_ask_ex"] = np.log1p(data["ask_ex"])
ax = sns.histplot(data, x="log_ask_ex", bins=50)  # type: ignore
ax.title.set_text(f"Histogram of ask exchange (log1p)")


In [ ]:
plot_kde_target("log_ask_ex", clip=[0, 5])


**Observation:**
* Applying a $\log$ transform leads to a easily distinguishable distribution

#### Bid

In [ ]:
ax = sns.histplot(data, x="bid_ex", bins=50)  # type: ignore
ax.title.set_text("Histogram of bid (exchange)")


In [ ]:
data["log_bid_ex"] = np.log1p(data["bid_ex"])
ax = sns.histplot(data, x="log_bid_ex", bins=50)  # type: ignore
ax.title.set_text(f"Histogram of bid exchange (log1p)")


In [ ]:
plot_kde_target("log_bid_ex", clip=[0, 5])


In [ ]:
data["log_bid_ex"] = np.log1p(data["bid_ex"])
ax = sns.histplot(data, x="log_bid_ex", bins=50)  # type: ignore
ax.title.set_text("Histogram of bid exchange (log1p)")


In [ ]:
plot_kde_target("log_bid_ex", clip=[-5, 6])


**Observation:**
- One can choose different constants, but small constants, e. g., `const=1e-2` gives fuzzy, yet distributions that are easier to distinguish. Also note the higher correlation with the target. 


**Observation:**
- log on size seems to worsen results.
- `TODO:` investigate further, what the reason is. e. g., how many outliers...

# NaNs 🪲

In [ ]:
def visualize_nan():
    """
    Visualize NaN values in a heatmap to learn about patterns.
    """
    plt.subplots()
    sns.heatmap(data.head(50).isnull(), cbar=False)
    plt.xlabel("feature")
    plt.ylabel("row")
    plt.title("Missing values (colored in bright beige)")
    plt.show()


In [ ]:
visualize_nan()


In [ ]:
isna_vals = data.isna().sum().sort_values(ascending=False)
isna_vals = isna_vals.loc[lambda x: x > 0]

ax = isna_vals.T.plot(
    kind="bar",
    figsize=ratio,
    legend=False,
    xlabel="No. of missing values",
    ylabel="feature",
    title="Missing values",
)


In [ ]:
isna_vals_over_time = (
    data[isna_vals.index.tolist()]
    .groupby(data["QUOTE_DATETIME"].dt.date)
    .agg(lambda x: x.isnull().sum())
)
isna_vals_over_time.plot(
    kind="line",
    figsize=ratio,
    title="Missing values over time",
    xlabel="Timestamp",
    ylabel="No. of missing values",
)


In [ ]:
# adapted from: https://github.com/ResidentMario/missingno/blob/master/missingno/missingno.py

isna_data = data.iloc[
    :, [i for i, n in enumerate(np.var(data.isnull(), axis="rows")) if n > 0]
]

corr_mat = isna_data.isnull().corr()
mask = np.zeros_like(corr_mat)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(9, 9))
ax = sns.heatmap(corr_mat, mask=mask, annot=False, annot_kws={"size": 10}, ax=ax)
ax.title.set_text("Correlation between missing features")


In [ ]:
# TODO: Check if there is a pattern between the missing values


**Observation:**
- Note that also important features like `price_all_lead` or `price_ex_lag` are missing. This has an impact, whether it is possible to calculate the classical rules like the trade rule. Consider this when reporting results.
- Missing values become more of a problem towards the end of the data set. At the same time the number of trades also increases.

# Correlations of engineered features 🎲

### Correlations 🎲

In [ ]:
corr = data.corr()

sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values)  # type: ignore


### Correlation with target 🎲

In [ ]:
sort_criteria = corr["buy_sell"].abs().sort_values(ascending=False)
corr_target = corr.sort_values("buy_sell", ascending=False)["buy_sell"]
corr_target.loc[sort_criteria.index].to_frame()


In [ ]:
# adapted from here: https://www.kaggle.com/code/willkoehrsen/featuretools-for-good

# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.975)]

print(to_drop)


**Observation:**
- Newly engineered features have a high correlation with the target, which is positive. Investigate if the correlation is due to randomness or whether a pattern is observable.
- `_ex` and `BEST_` features seem to be rather redundant, even if a high threshold for correlation is chosen.

### Collinearity of features🎲

In [ ]:
# Set the threshold
threshold = 0.975

# Empty dictionary to hold correlated variables
above_threshold_vars = {}

# For each column, record the variables that are above the threshold
for col in corr:
    above_threshold_vars[col] = list(corr.index[corr[col] > threshold])

pd.Series(above_threshold_vars)


# Analyses of engineered features

In [ ]:
corr_target.loc[sort_criteria.index].to_frame().T


### What works ✔️

#### Trade price vs distance from mid to ask

Similar to `EMO`, where the applied rule depends on whether the trade is at the ask or bid. Similarily `CLVN` uses percentage bounds e. g., $~20~\%$ of spread.

In [ ]:
plot_kde_target("rel_ask_ex", clip=[-2, 2])


In [ ]:
plot_kde_target_comparsion("rel_ask_ex", years=[2006, 2010, 2013], clip=[-2, 2])


#### Depth

Implicitly used in depth rule. Depth rule would assign a buy if `bid_ask_size_ratio_ex` is above one and a sell if it is below zero. But only used for mid-spread trades.

In [ ]:
plot_kde_target("bid_ask_size_ratio_ex", clip=[0, 100])


In [ ]:
plot_kde_target_comparsion(
    "bid_ask_size_ratio_ex", years=[2006, 2010, 2013], clip=[0, 100]
)


#### $\log$ bid ex

In [ ]:
plot_kde_target("log_bid_ex")


In [ ]:
plot_kde_target_comparsion("log_bid_ex", years=[2006, 2010, 2013])


**Observation:**
- Application of the $\log$ leads to highly differentiable distributions, that remain stable over time.

#### $\log$ trade price

In [ ]:
plot_kde_target("log_trade_price")


In [ ]:
plot_kde_target_comparsion("log_trade_price", years=[2006, 2010, 2013])


**Observation:**
- Application of the $\log$ leads to highly differentiable distributions, that remain stable over time.

#### Ask size

In [ ]:
plot_kde_target("ask_size_ex", clip=[0, 2000])


In [ ]:
plot_kde_target_comparsion("ask_size_ex", years=[2006, 2010, 2013], clip=[0, 2000])


#### Bid size

In [ ]:
plot_kde_target("bid_size_ex", clip=[0, 1000])


In [ ]:
plot_kde_target_comparsion("bid_size_ex", years=[2006, 2010, 2013], clip=[0, 1000])


#### Quote rule

In [ ]:
plot_kde_target("abs_mid_ex", clip=[-0.5, 0.5])


In [ ]:
plot_kde_target_comparsion("abs_mid_ex", years=[2006, 2010, 2013], clip=[-0.5, 0.5])


**Observation:**
- Compared with tick rule or reverse tick rule, quote rule is the only classical rule, where distributions are somewhat distinguishable
- On can clearly see that the quote rule works better at the beginning of the data set and its performance worsens over time.

#### Day of the month

In [ ]:
plot_kde_target("day")


In [ ]:
plot_kde_target("date_year_min")


**Observation:**
* Judging from the plot there seems to be a seasonal pattern e. g., more buys 
at the beginning of the month and more sells towards the end of the month. 
* Due to the distributions it could make sense to include date features in some feature sets. But do not include in the most basic data set.

## What doesn't ❌

#### tick rule

In [ ]:
plot_kde_target("chg_lead_ex", clip=[-5, 5])


In [ ]:
plot_kde_target_comparsion("chg_lead_ex", years=[2006, 2010, 2013], clip=[-5, 5])


**Observation:**
- Distributions are hardly distinguishable. 
- Results seem to worsen over time, which is consistent to the observations of Grauer et. al.

#### reverse tick rule

In [ ]:
plot_kde_target_comparsion("chg_lag_ex", years=[2006, 2010, 2013], clip=[-5, 5])


In [ ]:
plot_kde_target_comparsion("chg_lag_ex", years=[2006, 2010, 2013], clip=[-5, 5])


**Observation:**
- Distributions are hardly distinguishable. 
- Results worsen over time.

# Impact of scaling 🔢

In [ ]:
plot_kde_target("log_bid_ex", clip=[-5, 8])


In [ ]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
scaler = StandardScaler()
data["log_bid_ex_scaled"] = scaler.fit_transform(
    X=data["log_bid_ex"].values.reshape(-1, 1)
)


In [ ]:
data["log_bid_ex_scaled"].describe()


In [ ]:
plot_kde_target("log_bid_ex_scaled", clip=[-5, 5])


In [ ]:
del data


# Cross-validation⛑️

In [ ]:
oe_option_type = OrdinalEncoder(
    unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
)
oe_root = OrdinalEncoder(
    unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
)
oe_issue_type = OrdinalEncoder(
    unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
)


In [ ]:
def transform(data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:

    # date features
    x = pd.DataFrame(
        data={"date_year": data["QUOTE_DATETIME"].dt.year}, index=data.index
    )

    x["date_month_sin"] = np.sin(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)
    x["date_month_cos"] = np.cos(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)

    seconds_in_day = 24 * 60 * 60
    seconds = (
        data["QUOTE_DATETIME"] - data["QUOTE_DATETIME"].dt.normalize()
    ).dt.total_seconds()

    x["date_time_sin"] = np.sin(2 * np.pi * seconds / seconds_in_day)
    x["date_time_cos"] = np.cos(2 * np.pi * seconds / seconds_in_day)

    # option features
    x["ttm"] = (
        data["EXPIRATION"].dt.to_period("M") - data["QUOTE_DATETIME"].dt.to_period("M")
    ).apply(lambda x: x.n)
    x[["myn", "day_vol"]] = data[["myn", "day_vol"]]
    x["log_strk_prc"] = np.log1p(data["STRK_PRC"])

    # binarize
    # "bin_OPTION_TYPE", "bin_issue_type", "bin_ROOT",

    # size features
    x["bid_ask_size_ratio_ex"] = data["bid_size_ex"] / data["ask_size_ex"]
    x["rel_bid_size_ex"] = data["TRADE_SIZE"] / data["bid_size_ex"]
    x["rel_ask_size_ex"] = data["TRADE_SIZE"] / data["ask_size_ex"]
    x[["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]] = data[
        ["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]
    ]

    # classical
    mid_ex = 0.5 * (data["ask_ex"] + data["bid_ex"])
    mid_best = 0.5 * (data["BEST_ASK"] + data["BEST_BID"])
    x["rel_ask_ex"] = (data["TRADE_PRICE"] - mid_ex) / (data["ask_ex"] - mid_ex)
    x["rel_bid_ex"] = (mid_ex - data["TRADE_PRICE"]) / (mid_ex - data["bid_ex"])
    x["BEST_rel_bid"] = (data["TRADE_PRICE"] - mid_best) / (data["BEST_ASK"] - mid_best)
    x["BEST_rel_ask"] = (mid_best - data["TRADE_PRICE"]) / (mid_best - data["BEST_BID"])
    x["bid_ask_ratio_ex"] = data["bid_ex"] / data["ask_ex"]

    x["chg_ex_lead"] = data["TRADE_PRICE"] - data["price_ex_lead"]
    x["chg_ex_lag"] = data["TRADE_PRICE"] - data["price_ex_lag"]
    x["chg_all_lead"] = data["TRADE_PRICE"] - data["price_all_lead"]
    x["chg_all_lag"] = data["TRADE_PRICE"] - data["price_all_lag"]

    x[
        [
            "log_ask_ex",
            "log_bid_ex",
            "log_BEST_ASK",
            "log_BEST_BID",
            "log_trade_price",
            "log_price_all_lag",
            "log_price_all_lead",
            "log_price_ex_lag",
            "log_price_ex_lead",
        ]
    ] = np.log1p(
        data[
            [
                "ask_ex",
                "bid_ex",
                "BEST_ASK",
                "BEST_BID",
                "TRADE_PRICE",
                "price_all_lag",
                "price_all_lead",
                "price_ex_lag",
                "price_ex_lead",
            ]
        ]
    )

    # https://stackoverflow.com/questions/70727291/how-do-i-know-whether-a-sklearn-scaler-is-already-fitted-or-not

    if not hasattr(oe_option_type, "n_features_in_"):
        oe_option_type.fit(data["OPTION_TYPE"].astype(str).values.reshape(-1, 1))
    x["bin_option_type"] = oe_option_type.transform(
        data["OPTION_TYPE"].astype(str).values.reshape(-1, 1)
    )

    if not hasattr(oe_root, "n_features_in_"):
        oe_root.fit(data["ROOT"].astype(str).values.reshape(-1, 1))
    x["bin_root"] = oe_root.transform(data["ROOT"].astype(str).values.reshape(-1, 1))

    if not hasattr(oe_issue_type, "n_features_in_"):
        oe_issue_type.fit(data["issue_type"].astype(str).values.reshape(-1, 1))
    x["bin_issue_type"] = oe_issue_type.transform(
        data["issue_type"].astype(str).values.reshape(-1, 1)
    )

    x.replace([np.inf, -np.inf], np.nan, inplace=True)

    y = data["buy_sell"]
    return x, y


In [ ]:
train = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_extended_60.parquet"
)
x_train, y_train = transform(train)

del train


In [ ]:
val = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_extended_20.parquet"
)
x_val, y_val = transform(val)

del val


In [ ]:
test = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_extended_20.parquet"
)
x_test, y_test = transform(test)

del test


In [ ]:
classical_features = [
    "BEST_rel_bid",
    "BEST_rel_ask",
    "rel_ask_ex",
    "rel_bid_ex",
    "bid_ask_ratio_ex",
    "log_ask_ex",
    "log_bid_ex",
    "log_BEST_ASK",
    "log_BEST_BID",
    "chg_ex_lag",
    "chg_ex_lead",
    "chg_all_lag",
    "chg_all_lead",
    "log_trade_price",
    "log_price_all_lag",
    "log_price_all_lead",
    "log_price_ex_lag",
    "log_price_ex_lead",
]

size_features = [
    "TRADE_SIZE",
    "bid_ask_size_ratio_ex",
    "rel_bid_size_ex",
    "rel_ask_size_ex",
    "bid_size_ex",
    "ask_size_ex",
]
option_features = [
    "bin_option_type",
    "bin_issue_type",
    "bin_root",
    "myn",
    "log_strk_prc",
    "ttm",
    "day_vol",
]
date_features = [
    "date_time_cos",
    "date_time_sin",
    "date_month_cos",
    "date_month_sin",
    "date_year",
]

cat_features = ["bin_root", "bin_issue_type", "bin_option_type"]


In [ ]:
def evaluate(
    features: List[str], cat_features: Optional[List[str]]
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    params = {
        "od_type": "Iter",
        "logging_level": "Silent",
        "depth": 8,
        "loss_function": "Logloss",
        "task_type": "GPU",
        "cat_features": cat_features,
        "random_seed": 42,
    }

    res = cross_validate(
        CatBoostClassifier(**params),
        x_train[features],
        y_train,
        cv=3,
        return_estimator=True,
    )
    results_cv = pd.DataFrame(res["test_score"], columns=["test_score_cv"])
    print(res)

    oos = []
    feature_importances = []
    for i, model in enumerate(res["estimator"]):

        oos.append(
            [
                i,
                model.score(x_train[features], y_train),
                model.score(x_val[features], y_val),
                model.score(x_test[features], y_test),
            ]
        )

        feature_importance = model.get_feature_importance(prettified=True).add_prefix(
            f"fold_{i}_"
        )
        feature_importances.append(feature_importance)

    results_oos = pd.DataFrame(
        data=oos, columns=["fold", "acc_train", "acc_val", "acc_test"]
    )
    results_fi = pd.concat(feature_importances, axis=1)

    return results_cv, results_oos, results_fi


In [ ]:
results_cv, results_oos, results_fi = evaluate(classical_features, [])


In [ ]:
results_cv


In [ ]:
results_oos


In [ ]:
results_fi


In [ ]:
results_cv, results_oos, results_fi = evaluate(
    [*classical_features, *size_features], []
)


In [ ]:
results_cv


In [ ]:
results_oos


In [ ]:
results_fi


In [ ]:
results_cv, results_oos, results_fi = evaluate(
    [*classical_features, *size_features, *date_features, *option_features],
    cat_features,
)


In [ ]:
results_cv


In [ ]:
results_oos


In [ ]:
results_fi


# Conclusion 🔗

**Observation:**
- log 
- Features of Grauer et. al seem to work
- Binning gives mixed results e. g., for trade size and ttm
- Highly correlated columns don't matter 

- log on size seems to worsen results, but improves results for prices
- `TODO:` investigate further, what the reason is. e. g., some skewness, but outliers...
- `TODO:` investigate further, if there is an economic intuition behind it
- Classical features have hardly any importance. But keep them for comparsion
- Features of Grauer et. al seem to work
- Binning gives mixed results e. g., for trade size and ttm
- `TODO:` update [feature proposal](https://github.com/KarelZe/thesis/blob/main/references/obsidian/%F0%9F%8D%ACImplementation/%F0%9F%A7%AAFeature%20Engineering/%F0%9F%A7%83Feature%20Sets.md) accordingly
- `TODO:` Correlation is a common method for feature selection https://en.wikipedia.org/wiki/Feature_selection#Filter_method
- `TODO:` Remove highly correlated columns if possible
- `TODO:` Results are not suprising, when compared with feature importances of first gbm (taken from [this notebook](https://github.com/KarelZe/thesis/blob/d0e078a8030e1ba47b761b4a1abfc699a629ca0a/notebooks/3.0-mb-feature_engineering_baseline.ipynb)):
